---
description: action list
output-file: fnctions.html
title: functions

---

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp functions

In [ ]:
#| export
from pathlib import Path

def get_absolute_path(path):
    name = Path(path)
    return name.absolute().as_posix()

In [ ]:
#| export 
import importlib.resources

def get_dll_path(package_name, dll_filename):
    try:
        with importlib.resources.path(package_name, dll_filename) as dll_path:
            return str(dll_path)
    except FileNotFoundError as e:
        raise FileNotFoundError(f"The DLL file '{dll_filename}' was not found in the package '{package_name}'.") from e


In [ ]:
#| export

import winreg
import os


def add_dll_to_registry(dll_path, key_path):
    try:
        # Connect to the registry and open the specified key
        registry_key = winreg.OpenKey(winreg.HKEY_CURRENT_USER, key_path, 0, winreg.KEY_SET_VALUE)

        # Set the value for the new registry entry as a string (REG_SZ)
        winreg.SetValueEx(registry_key, "FilePathCheckerModule", 0, winreg.REG_SZ, dll_path)

        # Close the registry key
        winreg.CloseKey(registry_key)
        print("DLL path added to registry as a string value successfully.")
    except WindowsError as e:
        print("Error while adding DLL path to registry: ", e)
        
        
def get_registry_value(key_path, value_name):
    try:
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, key_path) as key:
            value, value_type = winreg.QueryValueEx(key, value_name)
            return value
    except FileNotFoundError:
        return None


In [ ]:
#| export

def check_dll():
    dll_path = get_dll_path("HwpApi", "FilePathCheckerModuleExample.dll")
    key_path = "SOFTWARE\\HNC\\HwpAutomation\\Modules"
    value_name = "FilePathCheckerModule" 

    value = get_registry_value(key_path, value_name)

    if value is None:
        add_dll_to_registry(dll_path, key_path)
    return True


In [ ]:
#| export 

def get_value(dict_, key):
    try:
        return dict_[key]
    except KeyError:
        raise KeyError(f"{key}를 해당하는 키 중 찾을 수 없습니다. 키는 {', '.join(dict_.keys())} 중에 있어야 합니다.")

In [ ]:
#| export

def set_charshape(charshape, fontname:str, font_type:int, fontcolor, fontsize:int, bold:bool, intalic:bool, strike_out, under_line, fontratio):
    params = []
    categories = ["Hangul", "Latin", "Hanja", "Japanese", "Other", "User"]
    
    params += [("FaceName"+cat, fontname) for cat in categories] if fontname else []
    params += [("FontType"+cat, fonttype) for cat in categories] if fontname else []
    params += [("Ratio"+cat, fontratio) for cat in categories] if fontratio else []
    
    params += list(
        filter(lambda x: x[1], 
            [
                ("TextColor", fontcolor),
                ("Height", app.api.PointToHwpUnit(fontsize) if old_fontsize else None),
                ("Bold", bold),
                ("Italic", italic),
                ("StrikeOutType", strike_out),
                ("UnderlineType", underline),
            ]
        )
    ) 
    for key, value in params:
            setattr(charshape, key, value)
    
    return charshape


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()